## 모듈화

In [24]:
import tensorflow as tf
import numpy as np
from datetime import datetime

In [25]:
n_features = 3

X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

w1 = tf.Variable(tf.random_normal((n_features, 1)), name='weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name='weights2')
b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')

relu1 = tf.maximum(z1, 0, name='relu1')
relu2 = tf.maximum(z2, 0, name='relu2')

output = tf.add(relu1, relu2, name='output')

반복적인 코드는 유지보수하기 어렵고 에러가 발생하기 쉽다
텐서플로에서는 DRY(dont repeat yourself)원칙을 유지하도록 도와준다.

In [26]:
def relu(X):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        W = tf.Variable(tf.random_normal(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, W), b, name='z')
        return tf.maximum(z, 0, name='relu')

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]

output = tf.add_n(relus, name='output')

In [37]:
x_train = tf.random_uniform((2000, n_features), -1, 1)
y_train = tf.random_uniform((2000, 1), 0, 5)
print(x_train.shape, y_train.shape)

rows, cols = x_train.shape
rows, cols = int(rows), int(cols)

(2000, 3) (2000, 1)


In [38]:
int(rows)

2000

In [39]:
# 학습 설정값
n_epochs = 300
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(int(rows)/batch_size))

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = "{}/run-{}".format(root_logdir, now)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) # FileWriter 객체를 만들어 로그 디렉터리에 있는 로그 파일에 서머리를 기록한다.

In [40]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch* n_batches + batch_index)
    indices = np.random.randint(rows, size=batch_size)
    X_batch = x_train[indices]
    y_batch = y_train[indices]
    return X_batch, y_batch

In [41]:
init = tf.global_variables_initializer() # 변수 초기화
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
            
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 1 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE", mse.eval(feed_dict={X: X_batch, y: y_batch}))
            save_path = saver.save(sess, './my_naming_model_{}.ckpt'.format(epoch))
    
    best_theta = theta.eval()
    save_path = saver.save(sess, './my_naming_model_final.ckpt'.format(epoch))
    file_writer.close()

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([ 684,  559, 1653, 1216,  835,  763, 1731, 1383, 1033, 1747,  277,
       1778, 1828,  599, 1094, 1496,  600, 1420,  314,  705, 1510,  551,
         87,  174, 1624,  849, 1701,  537,  845,   72,  777, 1940,  115,
        976,  755, 1733, 1871, 1455,  448,  850,   99, 1201,  755,  797,
        659, 1171,  910,  423, 1312, 1985, 1289,  697,  639,  544,  543,
        714,  244,  151,  675,  510, 1483,  882, 1207,   28,  802,  128,
        128, 1956, 1077, 1925, 1574, 1512,  756,  273,  335,  388, 1641,
       1066, 1466,  543,  888,  257, 1345, 1961,   57,  291, 1894, 1143,
        779,  430, 1106,   91, 1920,  398,  611, 1589,  908,  633, 1962,
         84])